In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import os

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from src.utils.time_series_procs import split_sequences
from src.models.transformer_ts_tf import Transformer

In [2]:
def get_times_series_data(df, cut_date):
    columns = list(df.columns.drop(["R28D"]))
    columns.append("R28D")
    df = df.reindex(columns=columns)
    df["Data/Hora"] = pd.to_datetime(df["Data/Hora"])
    df_train = df[df["Data/Hora"] < cut_date]
    df_test = df[df["Data/Hora"] >= cut_date]
    df_train = df_train.drop(["Data/Hora"], axis=1)
    df_test = df_test.drop(["Data/Hora"], axis=1)
    x_train, y_train = split_sequences(df_train.values, TIME_STEPS)
    x_test, y_test = split_sequences(df_test.values, TIME_STEPS)
    return x_train, y_train, x_test, y_test

In [3]:
class StandardScaler3DShape:
    def __init__(self):
        self.scaler = StandardScaler()

    def fit_transform(self, X, y=None):
        X_new = self.scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

    def fit(self, X, y=None):
        X_new = X.reshape(-1, X.shape[-1])
        self.scaler.fit(X_new)
        return self

    def transform(self, X, y=None):
        X_new = self.scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

In [4]:
def get_baseline_model(
    num_features, time_steps, num_heads=2, num_layers_enc=1, num_feed_forward=32
):
    model = Transformer(
        num_hid=num_features,
        time_steps=time_steps,
        num_head=num_heads,
        num_layers_enc=num_layers_enc,
        num_feed_forward=num_feed_forward,
    )

    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(
        optimizer=opt,
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [5]:
def make_repeated_time_series_k_fold(
    x,
    y,
    train_period,
    test_period,
    repeats=10,
    show_individual_results=True,
    show_final_result=True,
):
    results = []
    max_samples = x.shape[0]

    for _ in range(repeats):
        i = 0
        scores = []
        for _ in range(0, max_samples // train_period):
            x_train = x[i : i + train_period]
            y_train = y[i : i + train_period]
            x_test = x[i + train_period : i + train_period + test_period]
            y_test = y[i + train_period : i + train_period + test_period]
            i += train_period
            x_train = scaler.fit_transform(
                x_train.reshape(-1, x_train.shape[-1])
            ).reshape(x_train.shape)
            x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(
                x_test.shape
            )
            model = get_baseline_model(
                num_features=x_train.shape[-1], time_steps=x_train.shape[-2]
            )
            model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=0)
            rmse = model.evaluate(x_test, y_test, verbose=0)
            scores.append(rmse[1])
        results.append(scores)
        if show_individual_results:
            print("RMSE: %.3f (%.3f)" % (np.mean(scores), np.std(scores)))
    if show_final_result:
        print("\nRMSE: %.3f (%.3f)" % (np.mean(results), np.std(results)))
    return results

In [6]:
def make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats=10, timesteps_list=[3], show_results=True
):
    results = {}

    for timesteps in timesteps_list:
        x, y = split_sequences(df.values, timesteps)
        scores = make_repeated_time_series_k_fold(
            x,
            y,
            train_period,
            test_period,
            repeats=10,
            show_individual_results=False,
            show_final_result=False,
        )
        results[timesteps] = scores
        if show_results:
            print(
                "TIMESTEPS: %d RMSE: %.3f (%.3f)"
                % (timesteps, np.mean(scores), np.std(scores))
            )
    return results


<h3>Disable Tensorflow Warnings</h3>

In [7]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# FFT Transform

This notebook contains that experiments with the data transformed with fast fourier transform.

In [8]:
df_r3d_and_r7d = pd.read_csv(
    "../../../../data/interim/"
    + "r3d_and_r7d-no-resampling-with-fillna-and-fft-vars-only-"
    + "dados-historicos-partner_i-cement-CPIIE40.csv"
)
df_r3d_only = pd.read_csv(
    "../../../../data/interim/"
    + "r3d_only-no-resampling-with-fillna-and-fft-vars-only-"
    + "dados-historicos-partner_i-cement-CPIIE40.csv"
)
df_no_r3d_r7d = pd.read_csv(
    "../../../../data/interim/"
    + "no-r3d-r7d-no-resampling-with-fillna-and-fft-vars-only-"
    + "dados-historicos-partner_i-cement-CPIIE40.csv"
)

In [9]:
seed = 47
scaler = StandardScaler()

timesteps_list = [1, 3, 5, 7, 10, 15, 20]
repeats = 10

# Transformer Encoder

# Configs:

1. TIMESTEPS: [1, 3, 5, 7, 10, 15, 20]

2. Epochs: 100

3. Batch size: 32

4. num heads: 2

5. FF Layers enc: 1

      5.1 Units 32

6. Optimization Algorithm: Adam
    
    6.1. Learning Rate: 1e-2

# Times Series Repeated KFold Cross validation - different Timesteps values

Here we will make a Times Series KFold Cross validation with all 3 datasets. Here we are using the Blocking Time Series Split method. And with timeseries Split method. We also do it with different number of timesteps


<h3> Time Series Repeated KFold Cross Validation with vary timesteps - df_no_r3d_r7d</h3>

<b>Dataset: df_no_r3d_r7d:</b> In this dataset the R3D and R7D variables are not considered.

In [10]:
df = df_no_r3d_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora'], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [11]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [12]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.811 (0.397)
TIMESTEPS: 3 RMSE: 2.596 (0.625)
TIMESTEPS: 5 RMSE: 2.496 (0.491)
TIMESTEPS: 7 RMSE: 2.470 (0.529)
TIMESTEPS: 10 RMSE: 2.510 (0.416)
TIMESTEPS: 15 RMSE: 2.836 (0.572)
TIMESTEPS: 20 RMSE: 2.834 (0.485)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [13]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [14]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 3.010 (0.608)
TIMESTEPS: 3 RMSE: 3.368 (0.364)
TIMESTEPS: 5 RMSE: 3.582 (0.244)
TIMESTEPS: 7 RMSE: 3.581 (0.376)
TIMESTEPS: 10 RMSE: 2.997 (0.353)
TIMESTEPS: 15 RMSE: 3.255 (0.659)
TIMESTEPS: 20 RMSE: 2.852 (0.515)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [15]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [16]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.343 (0.499)
TIMESTEPS: 3 RMSE: 2.470 (0.791)
TIMESTEPS: 5 RMSE: 2.259 (0.310)
TIMESTEPS: 7 RMSE: 2.212 (0.341)
TIMESTEPS: 10 RMSE: 2.890 (0.706)
TIMESTEPS: 15 RMSE: 2.576 (0.726)
TIMESTEPS: 20 RMSE: 2.400 (1.098)


<h3> Time Series Repeated KFold Cross Validation - df_r3d_only</h3>

<b>Dataset: df_r3d_only:</b> In this dataset only R3D variable is considered.

In [17]:
df = df_r3d_only.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora'], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [18]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [19]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.775 (0.402)
TIMESTEPS: 3 RMSE: 2.672 (0.688)
TIMESTEPS: 5 RMSE: 2.660 (0.516)
TIMESTEPS: 7 RMSE: 2.465 (0.403)
TIMESTEPS: 10 RMSE: 2.573 (0.502)
TIMESTEPS: 15 RMSE: 2.904 (0.629)
TIMESTEPS: 20 RMSE: 3.132 (0.495)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [20]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [21]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 3.044 (0.477)
TIMESTEPS: 3 RMSE: 3.360 (0.391)
TIMESTEPS: 5 RMSE: 3.683 (0.268)
TIMESTEPS: 7 RMSE: 3.629 (0.333)
TIMESTEPS: 10 RMSE: 2.967 (0.277)
TIMESTEPS: 15 RMSE: 3.097 (0.391)
TIMESTEPS: 20 RMSE: 3.106 (0.437)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [22]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [23]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.182 (0.552)
TIMESTEPS: 3 RMSE: 2.408 (0.508)
TIMESTEPS: 5 RMSE: 2.212 (0.530)
TIMESTEPS: 7 RMSE: 2.361 (0.507)
TIMESTEPS: 10 RMSE: 2.685 (0.794)
TIMESTEPS: 15 RMSE: 2.641 (0.482)
TIMESTEPS: 20 RMSE: 4.510 (4.604)


<h3> Time Series Repeated KFold Cross Validation - df_r3d_and_r7d</h3>

<b>Dataset: df_r3d_and_r7d:</b> In this dataset both R3D and R7D variables are considered.

In [24]:
df = df_r3d_and_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora'], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [25]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [26]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.817 (0.384)
TIMESTEPS: 3 RMSE: 2.729 (0.642)
TIMESTEPS: 5 RMSE: 2.550 (0.473)
TIMESTEPS: 7 RMSE: 2.613 (0.901)
TIMESTEPS: 10 RMSE: 2.652 (0.451)
TIMESTEPS: 15 RMSE: 2.947 (0.628)
TIMESTEPS: 20 RMSE: 2.895 (0.473)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [27]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [28]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 3.114 (0.367)
TIMESTEPS: 3 RMSE: 3.337 (0.358)
TIMESTEPS: 5 RMSE: 3.388 (0.390)
TIMESTEPS: 7 RMSE: 3.610 (0.290)
TIMESTEPS: 10 RMSE: 3.061 (0.211)
TIMESTEPS: 15 RMSE: 3.506 (0.274)
TIMESTEPS: 20 RMSE: 2.760 (0.337)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [29]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [30]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.208 (0.483)
TIMESTEPS: 3 RMSE: 2.268 (0.432)
TIMESTEPS: 5 RMSE: 2.586 (0.608)
TIMESTEPS: 7 RMSE: 2.705 (0.955)
TIMESTEPS: 10 RMSE: 3.138 (1.996)
TIMESTEPS: 15 RMSE: 2.668 (0.570)
TIMESTEPS: 20 RMSE: 2.449 (0.602)
